In [1]:
load('packaging.rda', verbose=T)

Loading objects:
  save_functions
  set_parallel


### Data handling 

#### Restore factors **_data.frame_**

In [2]:
restore.factors <- function(x, reference, factor.vars=NULL){
    
    if (is.null(factor.vars)){
        factor.vars <- colnames(reference)[unlist(lapply(reference, is.factor))]
    }
    for (fct.var in factor.vars){
        if (!fct.var%in%colnames(x)){next()}
        if (!all(unique(x[,fct.var])%in%levels(reference[,fct.var]))){next()}
        x[,fct.var] <- factor(as.character(x[,fct.var]), levels=levels(reference[,fct.var]))
            
    }
    return(x)
}

#### Summarise by groups **_data.frame_**

In [3]:
summarise.metadata <- function(meta.data, group.vars, annotation.vars, 
                               keep_factors = T,
                               num.fun = mean, 
                               char.fun=function(x){paste(unique(as.character(x)), collapse=',')}){
    
    # copy data
    d <- meta.data
    
    require(dplyr)
    
    d %>% group_by_at(group.vars) %>% select_at(annotation.vars) %>% summarise_all(function(x){
        if (is.numeric(x)){num.fun(x)}else{char.fun(x)}
    }) %>% as.data.frame() -> sum.d
    
    if (keep_factors){
        sum.d <- restore.factors(x = sum.d, reference=meta.data)
    }
    # rownames(sum.d) <- apply(sum.d[,group.vars], 1, paste, collapse='_')
    return(sum.d)
    
}

In [4]:
cat(save_functions('data_handling'))

Loading required package: glue



restore.factors <- function (x, reference, factor.vars = NULL) 
{
    if (is.null(factor.vars)) {
        factor.vars <- colnames(reference)[unlist(lapply(reference, 
            is.factor))]
    }
    for (fct.var in factor.vars) {
        if (!fct.var %in% colnames(x)) {
            (next)()
        }
        if (!all(unique(x[, fct.var]) %in% levels(reference[, 
            fct.var]))) {
            (next)()
        }
        x[, fct.var] <- factor(as.character(x[, fct.var]), levels = levels(reference[, 
            fct.var]))
    }
    return(x)
}

save_functions <- function (basename = "functions", functions = NULL, envir.get = NULL, 
    R_script = T, Rda = T) 
{
    require(glue)
    if (is.null(envir.get)) {
        envir.get <- globalenv()
    }
    if (is.null(functions)) {
        env.vars <- sort(ls(envir = envir.get))
        functions <- env.vars[sapply(sapply(env.vars, get, envir = envir.get), 
            class) == "function"]
    }
    if (Rda) {
        save(list = fu